# Triangle of Circular Arcs - Problem 727

<p>Let $r_a$, $r_b$ and $r_c$ be the radii of three circles that are mutually and externally tangent to each other. The three circles then form a <dfn>triangle of circular arcs</dfn> between their tangency points as shown for the three blue circles in the picture below.</p>
<p></p>
<div class="center">
<img src="project/images/p727_circular_arcs.jpg" alt="CircularArcs" /></div>

<p></p>
Define the circumcircle of this triangle to be the red circle, with centre $D$, passing through their tangency points. Further define the incircle of this triangle to be the green circle, with centre $E$, that is mutually and externally tangent to all the three blue circles. Let $d=\vert DE \vert$ be the distance between the centres of the circumcircle and the incircle.
<p>
Let $\mathbb{E}(d)$ be the expected value of $d$ when $r_a$, $r_b$ and $r_c$ are integers chosen uniformly such that $1\leq r_a < r_b < r_c \leq 100$ and $\text{gcd}(r_a,r_b,r_c)=1$.</p>
<p>
Find $\mathbb{E}(d)$, rounded to eight places after the decimal point.</p>

## Solution.

In [1]:
from math import sqrt, gcd
import numpy as np

In [10]:
class CircularTriangle:
    def __init__(self, r_1, r_2, r_3):
        '''
        Input them in decreasing order - otherwise point A gets messed up!
        '''
        self.r_1 = r_1
        self.r_2 = r_2
        self.r_3 = r_3

    def r(self):
        '''
        Finds inradius given r_1, r_2, r_3.
        '''
        a = self.r_1 + self.r_2
        b = self.r_2 + self.r_3
        c = self.r_3 + self.r_1
    
        s = 0.5 * (a + b + c)
        A = sqrt(s * (s - a) * (s - b) * (s - c))
    
        r = A / s
        return r

    def B(self):
        return (-self.r_1, -self.r())

    def C(self):
        return (self.r_2, -self.r())

    def A(self):
        r_1 = self.r_1
        r_2 = self.r_2
        r_3 = self.r_3
        r = self.r()
        
        t_b = r / r_1
        t_g = -r / r_2

        t_2b = 2 * t_b / (1 - t_b**2)
        t_2g = 2 * t_g / (1 - t_g**2)

        x_b, y_b = self.B()
        x_c, y_c = self.C()

        x_a = (t_2g * x_c - t_2b * x_b) / (t_2g - t_2b)
        y_a = t_2b * (x_a - x_b) + y_b
        y_a2 = t_2g * (x_a - x_c) + y_c

        return (x_a, y_a)

    def mystery_point(self):
        x_a, y_a = self.A()
        x_b, y_b = self.B()
        x_c, y_c = self.C()

        r_1 = self.r_1
        r_2 = self.r_2
        r_3 = self.r_3
        r = self.r()


        # Use Newton's method
        def F(x, y):
            f_1 = sqrt((x - x_a)**2 + (y - y_a)**2) - sqrt((x - x_b)**2 + (y - y_b)**2) + (r_1 - r_3)
            f_2 = sqrt((x - x_a)**2 + (y - y_a)**2) - sqrt((x - x_c)**2 + (y - y_c)**2) + (r_2 - r_3)
            return np.matrix([f_1, f_2]).T
            

        def J(x, y):
            f_1x = (x-x_a) / sqrt((x - x_a)**2 + (y - y_a)**2) - (x-x_b) / sqrt((x - x_b)**2 + (y - y_b)**2) 
            f_2x = (x-x_a) / sqrt((x - x_a)**2 + (y - y_a)**2) - (x-x_c) / sqrt((x - x_c)**2 + (y - y_c)**2) 
            
            f_1y = (y-y_a) / sqrt((x - x_a)**2 + (y - y_a)**2) - (y-y_b) / sqrt((x - x_b)**2 + (y - y_b)**2) 
            f_2y = (y-y_a) / sqrt((x - x_a)**2 + (y - y_a)**2) - (y-y_c) / sqrt((x - x_c)**2 + (y - y_c)**2) 
            return np.matrix([[f_1x, f_1y], [f_2x, f_2y]])


        def update(x,y):
            return np.matrix([x,y]).T - J(x,y).I * F(x,y)

        x, y = 0.0, 0.0

        for _ in range(10):
            x, y = update(x, y).A1 
            if np.linalg.norm(F(x, y)) < 1e-12: 
                break

        return x, y

    def mystery_point_eff(self):
        x_a, y_a = self.A()
        x_b, y_b = self.B()
        x_c, y_c = self.C()

        r_1 = self.r_1
        r_2 = self.r_2
        r_3 = self.r_3
        r = self.r()

        # Use Descarte's Theorem to find r_4 - radius of Soddy circle
        k_1 = 1/r_1
        k_2 = 1/r_2
        k_3 = 1/r_3
        k_4 = k_1 + k_2 + k_3 + 2 * sqrt(k_1 * k_2 + k_2 * k_3 + k_3 * k_1)
        r_4 = 1 / k_4

        x =   (-r_1**2 + r_2**2 + (r_1+r_4)**2 - (r_2+r_4)**2)/(2*r_1 + 2*r_2)
        y = sqrt((r_1+r_4)**2 - (r_1+x)**2) - r

        return x, y

In [11]:
a = CircularTriangle(3,2,1)

In [12]:
a.mystery_point()

(np.float64(0.052173913043483434), np.float64(0.14782608695646468))

In [13]:
a.mystery_point_eff()

(0.052173913043478314, 0.1478260869565211)

In [15]:
def distance_from_origin(x, y):
    return sqrt(x**2 + y**2)

In [18]:
total = 0
n = 0

for r_1 in range(100, 2, -1):
    for r_2 in range(r_1-1, 1, -1):
        for r_3 in range(r_2-1, 0, -1):
            if gcd(r_1, r_2, r_3) == 1:
                triangle = CircularTriangle(r_1, r_2, r_3)
                x, y = triangle.mystery_point_eff()
                total += distance_from_origin(x, y)
                n += 1

In [19]:
print(total/n)

3.640391406429927


135739